# WP Post Views

At my Wordpress website I have a postviews plugin that calculates the number of reads per post. The final number of views is not only seen on the page, but also exposed with a standard Wordpress API.

Therefore, a number of useful stats can be calculated using this data. And that's what I do below.

In [1]:
%pip install plotly.express
%pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [82]:
# Read categories from API
import urllib.request, json 

pagesAvailable = 1
totalPosts = 0
fields = ""
page = 1

categories = []

while page <= pagesAvailable:
    addr = "https://sneakbug8.com/wp-json/wp/v2/categories?%s&per_page=100&page=%s" % (fields,page)
    print(addr)
    with urllib.request.urlopen(addr) as response:
        data = json.load(response)
        pagesAvailable = int(response.info()['x-wp-totalpages'])
        totalPosts = int(response.info()['x-wp-total'])

        categories = categories + data
    page += 1

print(len(categories))

with open('categories.json', 'w') as f:
    json.dump(categories, f)

https://sneakbug8.com/wp-json/wp/v2/categories?&per_page=100&page=1
23


In [77]:
# Read posts from API
import urllib.request, json 

pagesAvailable = 1
totalPosts = 0
fields = "_fields=id,title,link,views,date,categories"
page = 1

res = []

while page <= pagesAvailable:
    addr = "https://sneakbug8.com/wp-json/wp/v2/posts?%s&per_page=100&page=%s" % (fields,page)
    print(addr)
    with urllib.request.urlopen(addr) as response:
        data = json.load(response)
        pagesAvailable = int(response.info()['x-wp-totalpages'])
        totalPosts = int(response.info()['x-wp-total'])

        res = res + data
    page += 1

print(len(res))

with open('res.json', 'w') as f:
    json.dump(res, f)

https://sneakbug8.com/wp-json/wp/v2/posts?_fields=id,title,link,views,date,categories&per_page=100&page=1
https://sneakbug8.com/wp-json/wp/v2/posts?_fields=id,title,link,views,date,categories&per_page=100&page=2
https://sneakbug8.com/wp-json/wp/v2/posts?_fields=id,title,link,views,date,categories&per_page=100&page=3
https://sneakbug8.com/wp-json/wp/v2/posts?_fields=id,title,link,views,date,categories&per_page=100&page=4
308


In [87]:
import pandas as pd
import plotly.express as px

# df = pd.read_csv("./b.csv", sep=";")
df = pd.DataFrame(res)
df = df.set_index(["id"])
df["title"] = df["title"].map(lambda x: x["rendered"])
df.head()
# df[df["views"].isna()]

,id,date,link,title,categories,views
0,3964,2024-01-26T17:14:09,https://sneakbug8.com/relative-market-share/,What is a Relative Market Share (RMS),[319],6
1,3959,2024-01-25T12:49:37,https://sneakbug8.com/big-processes-in-bpmn/,💡 How to structure big processes in BPMN,[319],3
2,3943,2023-12-26T22:52:50,https://sneakbug8.com/top-3-mistakes-when-work...,Top 3 mistakes when working in English-speakin...,[56],58
3,3908,2023-11-19T23:06:25,https://sneakbug8.com/moscow-prioritization-me...,The MoSCoW prioritization method for Business ...,[87],63
4,3895,2023-10-19T10:51:58,https://sneakbug8.com/requirement-types/,Requirement Types,[319],101


In [85]:
# Fix missing views values

df[df["views"].isna()] = 0

In [88]:
dx = df.explode('categories').reset_index(drop=True)

dx = dx.groupby("categories").aggregate({"views":"sum","id":"count"})
dx = dx.rename(columns={"id": "post_count", "views":"total_views"})
dx["views_by_post"] = dx["total_views"] / dx["post_count"]

dx

,total_views,post_count,views_by_post
categories,,,
1,11528,42,274.476190
11,1479,6,246.500000
16,1350,4,337.500000
18,220,1,220.000000
21,2934,9,326.000000
56,10785,32,337.031250
87,6953,19,365.947368
187,9886,18,549.222222
189,6424,35,183.542857


In [90]:
# top read categories

import pandas as pd
import plotly.express as px

dc = pd.DataFrame(categories)
dc = dc.set_index(["id"])
dc = dc.join(dx).sort_values(by=['views_by_post'], ascending=False)

dc[["slug","total_views","post_count", "views_by_post"]]

,slug,total_views,post_count,views_by_post
id,,,,
187,project-management-ru,9886.0,18.0,549.222222
229,gamedev-ru,6361.0,14.0,454.357143
205,web-ru,24992.0,63.0,396.698413
231,movies-and-shows-ru,385.0,1.0,385.000000
87,project-management,6953.0,19.0,365.947368
16,web,1350.0,4.0,337.500000
56,softskills,10785.0,32.0,337.031250
21,gamedev,2934.0,9.0,326.000000
288,product-management,605.0,2.0,302.500000


In [91]:
# Visualize categories popularity
dc[["slug","total_views","post_count", "views_by_post"]]

dc = dc.sort_values(by=['views_by_post'], ascending=False)
fig = px.bar(dc, title='Views by post in a category', x="slug", y="views_by_post")
fig.show()

In [28]:
# View top 10 by views

dt = df.sort_values(by=['views'], ascending=False)
dt.head(10)

,id,link,title,views
178,2398,https://sneakbug8.com/ru/cookies-js-storages/,"{'rendered': 'Cookies, sessionStorage, localSt...",3058
159,2341,https://sneakbug8.com/ru/when-to-use-serif/,{'rendered': 'Когда использовать шрифт с засеч...,1570
75,2662,https://sneakbug8.com/ru/pmbok-7-izdanie-vyjde...,{'rendered': 'PMBOK 7: что нового?'},1544
98,2559,https://sneakbug8.com/ru/na-sluchaj-esli-kto-t...,{'rendered': 'На случай если кто то всё еще не...,1320
169,2370,https://sneakbug8.com/ru/opencart-full-guide-s...,{'rendered': 'OpenCart руководство: Simple &#8...,1268
173,2382,https://sneakbug8.com/ru/improve-conditionals-...,{'rendered': 'Советы по улучшению условий в Ja...,1161
62,3010,https://sneakbug8.com/six-project-management-c...,{'rendered': 'Six Project Management Constrain...,936
208,1401,https://sneakbug8.com/ru/unity-plavnoe-peremes...,{'rendered': 'Unity | Плавное перемещение каме...,930
76,2644,https://sneakbug8.com/ru/open-source-instrumen...,{'rendered': 'Open Source инструменты для упра...,904
281,563,https://sneakbug8.com/ru/rukovodstvo-po-unitys...,{'rendered': 'Руководство по UnityScript'},871


In [31]:
dt = df.sort_values(by=['date'], ascending=False)
fig = px.scatter(dt, title='Posts views trend', x="date", y="views")
fig.show()